In [2]:
import re
import sys
import os
import datetime
sys.path.append('E:\Study\my_github\smart regex')

import smart_regex as sr

# %load_ext line_profiler
%load_ext autoreload
%autoreload 2

In [3]:
patterns = """
    name | (姓 *名([一-龥*]{2,3}))
name | (姓 *名([a-z]{3,10}))
#name | ((([一-龥]|欧 *阳|独 *孤|令 *狐|皇 *甫|夏 *侯|呼 *延|诸 *葛|完 *颜|拓 *跋|公 *孙|宇 *文|北 *野|欧 *文|上 *官|端 *木|轩 *辕|慕 *蓉|公 *叔|司 *寇|百 *里|司 *马|北 *门|拓 *拔|慕 *容|独 *狐|爱 *新 *觉 *罗|尉 *迟|歐 *陽|叶 *赫 *纳 *拉|兀 *颜|司 *徒|耶 *律|单 *于|西 *门|公 *延|第 *五|令 *孤|北 *堂|蔚 *迟|西 *伯|申 *屠|公 *输)(先 *生|女 *士)))
email | ([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,4})
#phone | (((0086|86)-?)?(1(-| |–)*(3|4|5|6|7|8|9)((-| |–)*(0|1|2|3|4|5|6|7|8|9)(0|1|2|3|4|5|6|7|8|9)+)+))
phone_en | (((\+?\(?([17]|\d{2,3})?\(?[-.\r ()]{0,3}\d{2}\(?\d[-.\r\t ()]*\d{2,4}[-.\r\t ()]*\d{4,7}) *?))
emergent_contact | (((0086|86)-?)?(1[- –]*[3456789]([- –]*\d{1}){9}))
#gender | (女|软?妹 *子|萝 *莉|御 *姐|女 *王|菇 *凉|正 *太|大 *叔|鲜 *肉|男 *神|女 *神|暖 *男|male|female|([亚胜超冠若如润盼想来向])男)
age | ((\d {0,3}){2} {0,4}岁)
#height | ((0|1|2|3|4|5|6|7|8|9)+ *(cm|厘 *米))
height | (\d[.]\d{1,2} {0,4}(m|米))
#weight | ((0|1|2|3|4|5|6|7|8|9)(0|1|2|3|4|5|6|7|8|9)+(kg|公 *斤|斤))
#marital_status | (已 *婚|未 *婚|保 *密|(已|未)结 *婚)
#has_children | (已 *育)
birthday | ((((?P<ya>(\d {0,3}){4})年 {0,4}(?P<ma>(0?[1-9])|(1[0-2]))月 {0,4}((?P<da>(0?[1-9])|([1-2]\d)|(3[0-1]))日)?|(?P<yb> {0,3}19[5-9]\d {0,3}| {0,3}20[0-2]\d {0,3})[.．,/\-­–—―－一_\\、] {0,4}(?P<mb>(0?[1-9])|(1[0-2]))([.．,/\-­–—―－一_\\、] {0,4}(?P<db>(0?[1-9])|([1-2]\d)|(3[0-1])))?|((?P<dc>(0?[1-9])|([1-2]\d)|(3[0-1]))/ {0,3})?(?P<mc>(0?[1-9])|(1[0-2]))/ {0,3}(?P<yc>(19|20)(\d {0,3}){2})|(?P<yd> {0,3}19[5-9]\d {0,3}| {0,3}20[0-2]\d {0,3})(?P<md>(0?[1-9])|(1[0-2]))((?P<dd>(0?[1-9])|([1-2]\d)|(3[0-1])))?|(?P<me>(0?[1-9])|(1[0-2]))月/ {0,3}(?P<ye>(19|20)(\d {0,3}){2})年|(?P<mf>(0?[1-9])|(1[0-2]))/ {0,3}(?P<df>(0?[1-9])|([1-2]\d)|(3[0-1]))/ {0,3}(?P<yf> {0,3}19[5-9]\d {0,3}| {0,3}20[0-2]\d {0,3})|(?P<yx>(\d {0,3}){4})年 *度?|(?P<ybi>[01789]\d) {0,3}[.．,/\-­–—―－一_\\、年] {0,3}(?P<mbi>\d{1,2})月?|(?P<ybj>[789]\d) {0,3}年|(?P<yx1>(19[4-9]\d|20\d{2}))))出?生)
birthday | (出?生 *于(((?P<ya>(\d {0,3}){4})年 {0,4}(?P<ma>(0?[1-9])|(1[0-2]))月 {0,4}((?P<da>(0?[1-9])|([1-2]\d)|(3[0-1]))日)?|(?P<yb> {0,3}19[5-9]\d {0,3}| {0,3}20[0-2]\d {0,3})[.．,/\-­–—―－一_\\、] {0,4}(?P<mb>(0?[1-9])|(1[0-2]))([.．,/\-­–—―－一_\\、] {0,4}(?P<db>(0?[1-9])|([1-2]\d)|(3[0-1])))?|((?P<dc>(0?[1-9])|([1-2]\d)|(3[0-1]))/ {0,3})?(?P<mc>(0?[1-9])|(1[0-2]))/ {0,3}(?P<yc>(19|20)(\d {0,3}){2})|(?P<yd> {0,3}19[5-9]\d {0,3}| {0,3}20[0-2]\d {0,3})(?P<md>(0?[1-9])|(1[0-2]))((?P<dd>(0?[1-9])|([1-2]\d)|(3[0-1])))?|(?P<me>(0?[1-9])|(1[0-2]))月/ {0,3}(?P<ye>(19|20)(\d {0,3}){2})年|(?P<mf>(0?[1-9])|(1[0-2]))/ {0,3}(?P<df>(0?[1-9])|([1-2]\d)|(3[0-1]))/ {0,3}(?P<yf> {0,3}19[5-9]\d {0,3}| {0,3}20[0-2]\d {0,3})|(?P<yx>(\d {0,3}){4})年 *度?|(?P<ybi>[01789]\d) {0,3}[.．,/\-­–—―－一_\\、年] {0,3}(?P<mbi>\d{1,2})月?|(?P<ybj>[789]\d) {0,3}年|(?P<yx1>(19[4-9]\d|20\d{2})))))
#race | ((汉|壮|满|回|苗|维|维 *吾 *尔|土 *家|彝|蒙 *古|蒙|藏|布 *依|侗|瑶|朝 *鲜|白|哈 *尼|哈 *萨 *克|黎|傣)( *族))
#polit_status | ((党 *员|中 *共 *党 *员|共 *产 *党 *员|共 *产 *党 *党 *员|中 *国 *共 *产 *党 *党 *员|团 *员|共 *青 *团 *员|无 *党 *派 *人 *士|群 *众))
polit_status | (^|[^a-z0-9一-龥])(([一-龥])(党 *员|中 *共 *党 *员|共 *产 *党 *员|共 *产 *党 *党 *员|中 *国 *共 *产 *党 *党 *员|团 *员|共 *青 *团 *员|群 *众))($|[^a-z0-9一-龥])
#github | ((https?://)?(www.)?github[.]com/(^| \||;|.|/)+)
#github | ((https?://)?(^| \||;|.|/)+.github.io)
#linkedin | ((https?://)?(www.)?linkedin.com/in/(^| \||;|.|/)+)
#work_year | (((0|1|2|3|4|5|6|7|8|9)+(.|\|-|~)(0|1|2|3|4|5|6|7|8|9)|年(\+|以 *上)?)工 *作 *经 *验)
work_year | (([两一二三四五六七八九十\d]{1,3} {0,4}(年|个 *月)(\+|以 *上|半)?)[a-z一-龥]{0,5}工 *作 *经 *验)
$work_year | (应 *届(毕 *业)?生)
grad_time | ((((?P<ya>(\d {0,3}){4})年 {0,4}(?P<ma>(0?[1-9])|(1[0-2]))月 {0,4}((?P<da>(0?[1-9])|([1-2]\d)|(3[0-1]))日)?|(?P<yb> {0,3}19[5-9]\d {0,3}| {0,3}20[0-2]\d {0,3})[.．,/\-­–—―－一_\\、] {0,4}(?P<mb>(0?[1-9])|(1[0-2]))([.．,/\-­–—―－一_\\、] {0,4}(?P<db>(0?[1-9])|([1-2]\d)|(3[0-1])))?|((?P<dc>(0?[1-9])|([1-2]\d)|(3[0-1]))/ {0,3})?(?P<mc>(0?[1-9])|(1[0-2]))/ {0,3}(?P<yc>(19|20)(\d {0,3}){2})|(?P<yd> {0,3}19[5-9]\d {0,3}| {0,3}20[0-2]\d {0,3})(?P<md>(0?[1-9])|(1[0-2]))((?P<dd>(0?[1-9])|([1-2]\d)|(3[0-1])))?|(?P<me>(0?[1-9])|(1[0-2]))月/ {0,3}(?P<ye>(19|20)(\d {0,3}){2})年|(?P<mf>(0?[1-9])|(1[0-2]))/ {0,3}(?P<df>(0?[1-9])|([1-2]\d)|(3[0-1]))/ {0,3}(?P<yf> {0,3}19[5-9]\d {0,3}| {0,3}20[0-2]\d {0,3})|(?P<yx>(\d {0,3}){4})年 *度?))毕 *业)
#grad_time | ((0|1|2|3|4|5|6|7|8|9)(0|1|2|3|4|5|6|7|8|9)(0|1|2|3|4|5|6|7|8|9)(0|1|2|3|4|5|6|7|8|9)(届|级))
#college | ((大 *学|学 *院|学 *校|分 *校|中 *学|一 *中))
#degree | ((学 *士|bachelor|硕 *士|master|phd|双 *学 *士)|(中 *专|高 *中|high school|大 *专|专 *科|自 *考 *本 *科|成 *人 *本 *科|一 *本|二 *本|三 *本|本 *科|硕 *士|博 *士 *后|博 *后|博 *士|学 *士|双 *学 *士|研 *究 *生|mba|mpa|emba|cmba|大 *学 *肄 *业|undergraduate)(研 *究 *生)?(学 *位)?|(中 *技|高 *技))
degree | (^|[^a-z0-9一-龥])((小 *学|初 *中))($|[^a-z0-9一-龥])
major | (([a-z一-龥]{2,10})(专 *业|系)(?![一-龥]))
#constellation | ((摩 *羯|水 *瓶|双 *鱼|白 *羊|金 *牛|双 *子|巨 *蟹|狮 *子|处 *女|天 *秤|天 *蝎|射 *手)座?)
#blood_type | ((o|a|b|ab) *型)
email_subject | (?P<key>^民航资源网招聘频道用户) *[: ] *([^ |:]+)
edu_period | ((?P<vo_period>((?P<ya1>(\d {0,3}){4})[.．,/\-­–—―－一_\\、年] {0,3}(?P<ma1>(\d {0,3}){1,2})月? {0,3}[~\-­–—―－一至到]{1,6} {0,3}((?P<yb1>(\d {0,3}){4})[.．,/\-­–—―－一_\\、年] {0,3}(?P<mb1>(\d {0,3}){1,2})月?|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing|毕 *业 *时 *间)|(?P<ya2>(\d {0,3}){4}) {0,3}[.．,/\-­–—―－一_\\、]? ?(?P<ma2>(\d {0,3}){1,2})[.．,/\-­–—―－一_\\、]?(?P<da2>(\d {0,3}){1,2})? {0,3}[~\-­–—―－一至到]{1,6} {0,3}((?P<yb2>(\d {0,3}){4}) {0,3}[.．,/\-­–—―－一_\\、]? ?(?P<mb2>(\d {0,3}){1,2})[.．,/\-­–—―－一_\\、]?(?P<db2>(\d {0,3}){1,2})?|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ma3>(\d {0,3}){1,2})[.．,/\-­–—―－一_\\、,] {0,3}(?P<ya3>(\d {0,3}){4}) {0,3}[~\-­–—―－一至到]{1,6} {0,3}((?P<mb3>(\d {0,3}){1,2})[.．,/\-­–—―－一_\\、,] {0,3}(?P<yb3>(\d {0,3}){4})|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ya4>[12] {0,3}[09] {0,3}(\d {0,3}){2})年? {0,3}[~\-­–—―－一至到]{1,6} {0,3}((?P<yb4>[12] {0,3}[09] {0,3}(\d {0,3}){2})年?|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ya5>(\d {0,3}){4})[.．,/\-­–—―－一_\\、年] {0,3}(?P<ma5>(\d {0,3}){1,2})[.．,/\-­–—―－一_\\、月] {0,3}(?P<da5>(\d {0,3}){1,2})日? {0,3}[~\-­–—―－一至到]{1,6} {0,3}((?P<yb5>(\d {0,3}){4})[.．,/\-­–—―－一_\\、年] {0,3}(?P<mb5>(\d {0,3}){1,2})[.．,/\-­–—―－一_\\、月] {0,3}(?P<db5>(\d {0,3}){1,2})日?|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ya6>(\d {0,3}){4})年 {0,3}(?P<ma6>(\d {0,3}){1,2})月? {0,3}[~\-­–—―－一至到]{1,4} {0,3}(?P<mb6>(\d {0,3}){1,2}) {0,3}月|(?P<ya7>\d{4})[.．/](?P<ma7>\d{1,2}) {0,3}[~\-­–—―－一]{1,3} {0,3}(?P<mb7>\d{1,2})|(?P<ma10>(january|february|march|april|may|june|july|august|september|october|november|december|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec))[.．,/\-­–—―－一_\\、 ]{0,4}(?P<ya10>\d{4}) {0,3}([~\-­–—―－一]{1,4}|to) {0,3}((?P<mb10>(january|february|march|april|may|june|july|august|september|october|november|december|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec))[.．,/\-­–—―－一_\\、 ]{0,4}(?P<yb10>\d{4})|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ya11>\d{4})[.．,/\-­–—―－一_\\、 ]{0,4}(?P<ma11>(january|february|march|april|may|june|july|august|september|october|november|december|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)) {0,3}([~\-­–—―－一]{1,4}|to) {0,3}((?P<yb11>\d{4})[.．,/\-­–—―－一_\\、 ]{0,4}(?P<mb11>(january|february|march|april|may|june|july|august|september|october|november|december|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec))|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ya8>(\d {0,3}){4}) {0,3}年 {0,3}(?P<ma8>(\d {0,3}){1,2}) {0,3}月 {0,3}[~\-­–—―－一至到]{0,6} {0,3}((?P<yb8>(\d {0,3}){4}) {0,3}年 {0,3}(?P<mb8>(\d {0,3}){1,2}) {0,3}月?|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ma9>\d{2})[/](?P<ya9>\d{2,4})[-~至到]((?P<mb9>\d{2})/(?P<yb9>\d{2,4})|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ma22>\d{2})[/](?P<ya22>\d{4})[-~至到.．,/\-­–—―－一_\\、]((?P<yb22>\d{4})/(?P<mb22>\d{2})|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ya12> {0,3}19[5-9]\d {0,3}| {0,3}20[0-2]\d {0,3}). ?(?P<ma12>(0?[1-9])|(1[0-2]))(?P<yb12> {0,3}19[5-9]\d {0,3}| {0,3}20[0-2]\d {0,3}). ?(?P<mb12>(0?[1-9])|(1[0-2])))))
#edu_time | ((19|20)(0|1|2|3|4|5|6|7|8|9)(0|1|2|3|4|5|6|7|8|9)(届|级))
edu_time | ((((?P<ya>(\d {0,3}){4})年 {0,4}(?P<ma>(0?[1-9])|(1[0-2]))月 {0,4}((?P<da>(0?[1-9])|([1-2]\d)|(3[0-1]))日)?|(?P<yb> {0,3}19[5-9]\d {0,3}| {0,3}20[0-2]\d {0,3})[.．,/\-­–—―－一_\\、] {0,4}(?P<mb>(0?[1-9])|(1[0-2]))([.．,/\-­–—―－一_\\、] {0,4}(?P<db>(0?[1-9])|([1-2]\d)|(3[0-1])))?|((?P<dc>(0?[1-9])|([1-2]\d)|(3[0-1]))/ {0,3})?(?P<mc>(0?[1-9])|(1[0-2]))/ {0,3}(?P<yc>(19|20)(\d {0,3}){2})|(?P<yd> {0,3}19[5-9]\d {0,3}| {0,3}20[0-2]\d {0,3})(?P<md>(0?[1-9])|(1[0-2]))((?P<dd>(0?[1-9])|([1-2]\d)|(3[0-1])))?|(?P<me>(0?[1-9])|(1[0-2]))月/ {0,3}(?P<ye>(19|20)(\d {0,3}){2})年|(?P<mf>(0?[1-9])|(1[0-2]))/ {0,3}(?P<df>(0?[1-9])|([1-2]\d)|(3[0-1]))/ {0,3}(?P<yf> {0,3}19[5-9]\d {0,3}| {0,3}20[0-2]\d {0,3})|(?P<yx>(\d {0,3}){4})年 *度?))毕 *业)
#edu_time | ((19|20)(0|1|2|3|4|5|6|7|8|9)(0|1|2|3|4|5|6|7|8|9))
edu_college | (^|[^a-z0-9一-龥])([一-龥]+(大 *学|学 *院|学 *校|分 *校|中 *学|一 *中))($|[^a-z0-9一-龥])
edu_degree | ((学 *士|bachelor|硕 *士|master|phd|双 *学 *士)|(中 *专|高 *中|high school|大 *专|专 *科|自 *考 *本 *科|成 *人 *本 *科|一 *本|二 *本|三 *本|本 *科|硕 *士|博 *士 *后|博 *后|博 *士|学 *士|双 *学 *士|研 *究 *生|mba|mpa|emba|cmba|大 *学 *肄 *业|undergraduate)(研 *究 *生)?(学 *位)?|(中 *技|高 *技)(?![工术]))
edu_degree | (^|[^a-z0-9一-龥])((小 *学|初 *中))($|[^a-z0-9一-龥])
edu_major | (([a-z一-龥]{2,10})专 *业(?![一-龥]))
#edu_recruit | ((非?统 *招|自 *考|在 *职|在 *职 *研|在 *职 *硕 *士|交 *换 *学 *生|交 *换 *生|交 *流 *学 *生|交 *流 *生|在 *读|成 *教|成 *人 *教 *育|退 *学|函 *授|肄 *业|成 *人|专 *升 *本|脱 *产|网 *教|夜 *校|全 *日 *制|非 *全 *日 *制|保 *送))
edu_recruit | (^|[^a-z0-9一-龥])(交 *换|交 *流|自 *学)($|[^a-z0-9一-龥])
#expect_jnature | (全 *职|兼 *职|实 *习)
job_period | ((?P<vo_period>((?P<ya1>(\d {0,3}){4})[.．,/\-­–—―－一_\\、年] {0,3}(?P<ma1>(\d {0,3}){1,2})月? {0,3}[~\-­–—―－一至到]{1,6} {0,3}((?P<yb1>(\d {0,3}){4})[.．,/\-­–—―－一_\\、年] {0,3}(?P<mb1>(\d {0,3}){1,2})月?|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing|毕 *业 *时 *间)|(?P<ya2>(\d {0,3}){4}) {0,3}[.．,/\-­–—―－一_\\、]? ?(?P<ma2>(\d {0,3}){1,2})[.．,/\-­–—―－一_\\、]?(?P<da2>(\d {0,3}){1,2})? {0,3}[~\-­–—―－一至到]{1,6} {0,3}((?P<yb2>(\d {0,3}){4}) {0,3}[.．,/\-­–—―－一_\\、]? ?(?P<mb2>(\d {0,3}){1,2})[.．,/\-­–—―－一_\\、]?(?P<db2>(\d {0,3}){1,2})?|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ma3>(\d {0,3}){1,2})[.．,/\-­–—―－一_\\、,] {0,3}(?P<ya3>(\d {0,3}){4}) {0,3}[~\-­–—―－一至到]{1,6} {0,3}((?P<mb3>(\d {0,3}){1,2})[.．,/\-­–—―－一_\\、,] {0,3}(?P<yb3>(\d {0,3}){4})|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ya4>[12] {0,3}[09] {0,3}(\d {0,3}){2})年? {0,3}[~\-­–—―－一至到]{1,6} {0,3}((?P<yb4>[12] {0,3}[09] {0,3}(\d {0,3}){2})年?|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ya5>(\d {0,3}){4})[.．,/\-­–—―－一_\\、年] {0,3}(?P<ma5>(\d {0,3}){1,2})[.．,/\-­–—―－一_\\、月] {0,3}(?P<da5>(\d {0,3}){1,2})日? {0,3}[~\-­–—―－一至到]{1,6} {0,3}((?P<yb5>(\d {0,3}){4})[.．,/\-­–—―－一_\\、年] {0,3}(?P<mb5>(\d {0,3}){1,2})[.．,/\-­–—―－一_\\、月] {0,3}(?P<db5>(\d {0,3}){1,2})日?|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ya6>(\d {0,3}){4})年 {0,3}(?P<ma6>(\d {0,3}){1,2})月? {0,3}[~\-­–—―－一至到]{1,4} {0,3}(?P<mb6>(\d {0,3}){1,2}) {0,3}月|(?P<ya7>\d{4})[.．/](?P<ma7>\d{1,2}) {0,3}[~\-­–—―－一]{1,3} {0,3}(?P<mb7>\d{1,2})|(?P<ma10>(january|february|march|april|may|june|july|august|september|october|november|december|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec))[.．,/\-­–—―－一_\\、 ]{0,4}(?P<ya10>\d{4}) {0,3}([~\-­–—―－一]{1,4}|to) {0,3}((?P<mb10>(january|february|march|april|may|june|july|august|september|october|november|december|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec))[.．,/\-­–—―－一_\\、 ]{0,4}(?P<yb10>\d{4})|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ya11>\d{4})[.．,/\-­–—―－一_\\、 ]{0,4}(?P<ma11>(january|february|march|april|may|june|july|august|september|october|november|december|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)) {0,3}([~\-­–—―－一]{1,4}|to) {0,3}((?P<yb11>\d{4})[.．,/\-­–—―－一_\\、 ]{0,4}(?P<mb11>(january|february|march|april|may|june|july|august|september|october|november|december|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec))|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ya8>(\d {0,3}){4}) {0,3}年 {0,3}(?P<ma8>(\d {0,3}){1,2}) {0,3}月 {0,3}[~\-­–—―－一至到]{0,6} {0,3}((?P<yb8>(\d {0,3}){4}) {0,3}年 {0,3}(?P<mb8>(\d {0,3}){1,2}) {0,3}月?|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ma9>\d{2})[/](?P<ya9>\d{2,4})[-~至到]((?P<mb9>\d{2})/(?P<yb9>\d{2,4})|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ma22>\d{2})[/](?P<ya22>\d{4})[-~至到.．,/\-­–—―－一_\\、]((?P<yb22>\d{4})/(?P<mb22>\d{2})|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ya12> {0,3}19[5-9]\d {0,3}| {0,3}20[0-2]\d {0,3}). ?(?P<ma12>(0?[1-9])|(1[0-2]))(?P<yb12> {0,3}19[5-9]\d {0,3}| {0,3}20[0-2]\d {0,3}). ?(?P<mb12>(0?[1-9])|(1[0-2])))))
job_salary | (((大 *约|约)?(税 *前(月 *薪|年 *薪)?|税 *后(月 *薪|年 *薪)?|固 *薪|浮 *薪)?(￥|￥|人 *名 *币)?(大 *约|约)?(?P<va1>\d[\d,.]*) {0,4}(万|k|w)?( {0,4}[~\-­–—―－一至] {0,4}(?P<va2>\d[\d,.]*))? {0,4}(万|(k|w)(?![a-z]))(元|人 *民 *币|美 *元)?(以 *上)?(/天|/周|/月|/年|/m|/y|每 *年|每 *月)?|(大 *约|约)?(税 *前(月 *薪|年 *薪)?|税 *后(月 *薪|年 *薪)?|固 *薪|浮 *薪)?(￥|￥|人 *名 *币)?(大 *约|约)?(?P<vb1>\d[\d,.]*) {0,4}(万|k|w)?( {0,4}[~\-­–—―－一至] {0,4}(?P<vb2>\d[\d,.]*))? {0,4}(元|人 *民 *币|美 *元)?(以 *上)?(/天|/周|/月|/年|/m|/y|每 *年|每 *月)|(大 *约|约)?(税 *前(月 *薪|年 *薪)?|税 *后(月 *薪|年 *薪)?|固 *薪|浮 *薪)?(￥|￥|人 *名 *币)?(大 *约|约)?(?P<vc1>\d[\d,.]*) {0,4}(万|k|w)?( {0,4}[~\-­–—―－一至] {0,4}(?P<vc2>\d[\d,.]*))? {0,4}(以 *上)?(/月|/年|/m|/y|每 *年|每 *月)))
job_cpy | ((在|就|任|供)职 *于([a-z一-龥]{2,25}))
job_cpy | ((在|进 *入|工 *作 *于|在 *职 *于|就 *职 *于|任 *职 *于|供 *职 *于)([a-z一-龥()]{2,25}(公 *司|大 *学|医 *院|集 *团|研 *究 *院|网|银 *行|证 *券)))
job_cpy_nature | (^|[^a-z0-9一-龥])((上 *市|民 *营|国 *企|央 *企|外 *企|外 *资|美 *资|港 *资|台 *资|日 *企|韩 *企)(公 *司|企 *业))($|[^a-z0-9一-龥])
#job_cpy_nature | ((民 *营/私 *营)(公 *司|企 *业))
#job_cpy_size | (((0|1|2|3|4|5|6|7|8|9) *)+(~|\|-|­–|—|―|－|一|至) *((0|1|2|3|4|5|6|7|8|9) *)+人)
job_cpy_size | (^|[^a-z0-9一-龥])([a-z一-龥]{0,3}(\d+ {0,4}人(以 *上|以 *下)?))($|[^a-z0-9一-龥])
job_cpy_size | (\([a-z一-龥]{0,3}(\d+ {0,4}人(以 *上|以 *下)?)\))
job_cpy_size | (^|[^a-z0-9一-龥])((\d {0,4})+[~\-­–—―－一至] {0,4}(\d {0,4})+人)($|[^a-z0-9一-龥])
job_cpy_size | (\((\d {0,4})+[~\-­–—―－一至] {0,4}(\d {0,4})+人\))
job_dept | (^|[^a-z0-9一-龥])([a-z一-龥]{1,8}部)($|[^a-z0-9一-龥])
#job_dept | (所 *在 *部 *门 *:? *(未 *填 *写))
#job_nature | (全 *职|兼 *职|实 *习)
#job_cpy_financing_situation | ((a|b|c|d|e|a+|天 *使|天 *使+)轮)
#job_cpy_financing_situation | ((已|不 *需 *要|无 *需|未)融 *资)
#job_cpy_financing_situation | ((已|非|纳 *斯 *达 *克)上 *市)
#job_cpy_financing_situation | ((h|港|美)股)
proj_period | ((?P<vo_period>((?P<ya1>(\d {0,3}){4})[.．,/\-­–—―－一_\\、年] {0,3}(?P<ma1>(\d {0,3}){1,2})月? {0,3}[~\-­–—―－一至到]{1,6} {0,3}((?P<yb1>(\d {0,3}){4})[.．,/\-­–—―－一_\\、年] {0,3}(?P<mb1>(\d {0,3}){1,2})月?|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing|毕 *业 *时 *间)|(?P<ya2>(\d {0,3}){4}) {0,3}[.．,/\-­–—―－一_\\、]? ?(?P<ma2>(\d {0,3}){1,2})[.．,/\-­–—―－一_\\、]?(?P<da2>(\d {0,3}){1,2})? {0,3}[~\-­–—―－一至到]{1,6} {0,3}((?P<yb2>(\d {0,3}){4}) {0,3}[.．,/\-­–—―－一_\\、]? ?(?P<mb2>(\d {0,3}){1,2})[.．,/\-­–—―－一_\\、]?(?P<db2>(\d {0,3}){1,2})?|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ma3>(\d {0,3}){1,2})[.．,/\-­–—―－一_\\、,] {0,3}(?P<ya3>(\d {0,3}){4}) {0,3}[~\-­–—―－一至到]{1,6} {0,3}((?P<mb3>(\d {0,3}){1,2})[.．,/\-­–—―－一_\\、,] {0,3}(?P<yb3>(\d {0,3}){4})|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ya4>[12] {0,3}[09] {0,3}(\d {0,3}){2})年? {0,3}[~\-­–—―－一至到]{1,6} {0,3}((?P<yb4>[12] {0,3}[09] {0,3}(\d {0,3}){2})年?|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ya5>(\d {0,3}){4})[.．,/\-­–—―－一_\\、年] {0,3}(?P<ma5>(\d {0,3}){1,2})[.．,/\-­–—―－一_\\、月] {0,3}(?P<da5>(\d {0,3}){1,2})日? {0,3}[~\-­–—―－一至到]{1,6} {0,3}((?P<yb5>(\d {0,3}){4})[.．,/\-­–—―－一_\\、年] {0,3}(?P<mb5>(\d {0,3}){1,2})[.．,/\-­–—―－一_\\、月] {0,3}(?P<db5>(\d {0,3}){1,2})日?|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ya6>(\d {0,3}){4})年 {0,3}(?P<ma6>(\d {0,3}){1,2})月? {0,3}[~\-­–—―－一至到]{1,4} {0,3}(?P<mb6>(\d {0,3}){1,2}) {0,3}月|(?P<ya7>\d{4})[.．/](?P<ma7>\d{1,2}) {0,3}[~\-­–—―－一]{1,3} {0,3}(?P<mb7>\d{1,2})|(?P<ma10>(january|february|march|april|may|june|july|august|september|october|november|december|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec))[.．,/\-­–—―－一_\\、 ]{0,4}(?P<ya10>\d{4}) {0,3}([~\-­–—―－一]{1,4}|to) {0,3}((?P<mb10>(january|february|march|april|may|june|july|august|september|october|november|december|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec))[.．,/\-­–—―－一_\\、 ]{0,4}(?P<yb10>\d{4})|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ya11>\d{4})[.．,/\-­–—―－一_\\、 ]{0,4}(?P<ma11>(january|february|march|april|may|june|july|august|september|october|november|december|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)) {0,3}([~\-­–—―－一]{1,4}|to) {0,3}((?P<yb11>\d{4})[.．,/\-­–—―－一_\\、 ]{0,4}(?P<mb11>(january|february|march|april|may|june|july|august|september|october|november|december|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec))|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ya8>(\d {0,3}){4}) {0,3}年 {0,3}(?P<ma8>(\d {0,3}){1,2}) {0,3}月 {0,3}[~\-­–—―－一至到]{0,6} {0,3}((?P<yb8>(\d {0,3}){4}) {0,3}年 {0,3}(?P<mb8>(\d {0,3}){1,2}) {0,3}月?|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ma9>\d{2})[/](?P<ya9>\d{2,4})[-~至到]((?P<mb9>\d{2})/(?P<yb9>\d{2,4})|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ma22>\d{2})[/](?P<ya22>\d{4})[-~至到.．,/\-­–—―－一_\\、]((?P<yb22>\d{4})/(?P<mb22>\d{2})|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ya12> {0,3}19[5-9]\d {0,3}| {0,3}20[0-2]\d {0,3}). ?(?P<ma12>(0?[1-9])|(1[0-2]))(?P<yb12> {0,3}19[5-9]\d {0,3}| {0,3}20[0-2]\d {0,3}). ?(?P<mb12>(0?[1-9])|(1[0-2])))))
train_period | ((?P<vo_period>((?P<ya1>(\d {0,3}){4})[.．,/\-­–—―－一_\\、年] {0,3}(?P<ma1>(\d {0,3}){1,2})月? {0,3}[~\-­–—―－一至到]{1,6} {0,3}((?P<yb1>(\d {0,3}){4})[.．,/\-­–—―－一_\\、年] {0,3}(?P<mb1>(\d {0,3}){1,2})月?|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing|毕 *业 *时 *间)|(?P<ya2>(\d {0,3}){4}) {0,3}[.．,/\-­–—―－一_\\、]? ?(?P<ma2>(\d {0,3}){1,2})[.．,/\-­–—―－一_\\、]?(?P<da2>(\d {0,3}){1,2})? {0,3}[~\-­–—―－一至到]{1,6} {0,3}((?P<yb2>(\d {0,3}){4}) {0,3}[.．,/\-­–—―－一_\\、]? ?(?P<mb2>(\d {0,3}){1,2})[.．,/\-­–—―－一_\\、]?(?P<db2>(\d {0,3}){1,2})?|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ma3>(\d {0,3}){1,2})[.．,/\-­–—―－一_\\、,] {0,3}(?P<ya3>(\d {0,3}){4}) {0,3}[~\-­–—―－一至到]{1,6} {0,3}((?P<mb3>(\d {0,3}){1,2})[.．,/\-­–—―－一_\\、,] {0,3}(?P<yb3>(\d {0,3}){4})|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ya4>[12] {0,3}[09] {0,3}(\d {0,3}){2})年? {0,3}[~\-­–—―－一至到]{1,6} {0,3}((?P<yb4>[12] {0,3}[09] {0,3}(\d {0,3}){2})年?|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ya5>(\d {0,3}){4})[.．,/\-­–—―－一_\\、年] {0,3}(?P<ma5>(\d {0,3}){1,2})[.．,/\-­–—―－一_\\、月] {0,3}(?P<da5>(\d {0,3}){1,2})日? {0,3}[~\-­–—―－一至到]{1,6} {0,3}((?P<yb5>(\d {0,3}){4})[.．,/\-­–—―－一_\\、年] {0,3}(?P<mb5>(\d {0,3}){1,2})[.．,/\-­–—―－一_\\、月] {0,3}(?P<db5>(\d {0,3}){1,2})日?|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ya6>(\d {0,3}){4})年 {0,3}(?P<ma6>(\d {0,3}){1,2})月? {0,3}[~\-­–—―－一至到]{1,4} {0,3}(?P<mb6>(\d {0,3}){1,2}) {0,3}月|(?P<ya7>\d{4})[.．/](?P<ma7>\d{1,2}) {0,3}[~\-­–—―－一]{1,3} {0,3}(?P<mb7>\d{1,2})|(?P<ma10>(january|february|march|april|may|june|july|august|september|october|november|december|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec))[.．,/\-­–—―－一_\\、 ]{0,4}(?P<ya10>\d{4}) {0,3}([~\-­–—―－一]{1,4}|to) {0,3}((?P<mb10>(january|february|march|april|may|june|july|august|september|october|november|december|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec))[.．,/\-­–—―－一_\\、 ]{0,4}(?P<yb10>\d{4})|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ya11>\d{4})[.．,/\-­–—―－一_\\、 ]{0,4}(?P<ma11>(january|february|march|april|may|june|july|august|september|october|november|december|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)) {0,3}([~\-­–—―－一]{1,4}|to) {0,3}((?P<yb11>\d{4})[.．,/\-­–—―－一_\\、 ]{0,4}(?P<mb11>(january|february|march|april|may|june|july|august|september|october|november|december|jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec))|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ya8>(\d {0,3}){4}) {0,3}年 {0,3}(?P<ma8>(\d {0,3}){1,2}) {0,3}月 {0,3}[~\-­–—―－一至到]{0,6} {0,3}((?P<yb8>(\d {0,3}){4}) {0,3}年 {0,3}(?P<mb8>(\d {0,3}){1,2}) {0,3}月?|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ma9>\d{2})[/](?P<ya9>\d{2,4})[-~至到]((?P<mb9>\d{2})/(?P<yb9>\d{2,4})|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ma22>\d{2})[/](?P<ya22>\d{4})[-~至到.．,/\-­–—―－一_\\、]((?P<yb22>\d{4})/(?P<mb22>\d{2})|今|至 *今|现 *今|现 *在|当 *前|目 *前|now|current|present|ongoing)|(?P<ya12> {0,3}19[5-9]\d {0,3}| {0,3}20[0-2]\d {0,3}). ?(?P<ma12>(0?[1-9])|(1[0-2]))(?P<yb12> {0,3}19[5-9]\d {0,3}| {0,3}20[0-2]\d {0,3}). ?(?P<mb12>(0?[1-9])|(1[0-2])))))
"""
patterns = [[x.strip() for x in p.split('|', 1)] for p in patterns.split('\n') if p.startswith('#')]

In [4]:
re_patterns = [re.compile(pat[1]) for pat in patterns]

sr_patterns = []
for pat in patterns:
    try:
        sr_patterns.append((pat[0], sr.compile(pat[1])))
    except:
        print(pat)
sr_patterns = sr_patterns

['#phone', '(((0086|86)-?)?(1(-| |–)*(3|4|5|6|7|8|9)((-| |–)*(0|1|2|3|4|5|6|7|8|9)(0|1|2|3|4|5|6|7|8|9)+)+))']
['#job_cpy_size', '(((0|1|2|3|4|5|6|7|8|9) *)+(~|\\|-|\xad–|—|―|－|一|至) *((0|1|2|3|4|5|6|7|8|9) *)+人)']


In [5]:
def index_docs(docs=None):
    index = {}
    for doc_idx, doc in enumerate(docs):
        for line_idx, line in enumerate(doc):
            for token in sr.generate_ngram_chars(line, 2):
                if token not in index:
                    index[token] = set()
                index[token].add((doc_idx, line_idx))
    return index

In [8]:
folder = r'E:\Study\my_github\smart regex\test\test_data'
lines = []
for fname in os.listdir(folder):
    if fname.endswith('txt'):
        with open(os.path.join(folder, fname), encoding='utf-8') as f:
            lines.extend(l.strip() for l in f.readlines())
            
folder = r'E:\Study\my_github\smart regex\test\test_data'
docs = []
for fname in os.listdir(folder):
    if fname.endswith('txt'):
        with open(os.path.join(folder, fname), encoding='utf-8') as f:
            docs.append([l.strip() for l in f.readlines()])

In [9]:
%%timeit -n 1 -r 5
# 普通正则匹配时间
# re_results = []
for line in lines:
    for re_pat in re_patterns[:81]:
        mat = re_pat.search(line)
#         if mat:
#             re_results.append(mat.group())

88.8 ms ± 3.54 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [10]:
# %%timeit -n 1 -r 5
# smart regex 匹配时间
index = index_docs(docs)

sr_results_time = {}

for sr_pat in sr_patterns:
    now = datetime.datetime.now()
    mat = sr_pat[1].search(docs, index)
    sr_results_time[sr_pat] = (datetime.datetime.now() - now)

In [11]:
sorted(sr_results_time.items(), key=lambda x:x[1], reverse=True)

[(('#polit_status', <data_structs.Expression at 0x1c0a7f6ad30>),
  datetime.timedelta(seconds=1, microseconds=805306)),
 (('#name', <data_structs.Expression at 0x1c0a4dfadd8>),
  datetime.timedelta(microseconds=714093)),
 (('#edu_recruit', <data_structs.Expression at 0x1c0bbdf9470>),
  datetime.timedelta(microseconds=549556)),
 (('#degree', <data_structs.Expression at 0x1c0b2f54748>),
  datetime.timedelta(microseconds=494676)),
 (('#work_year', <data_structs.Expression at 0x1c0ab6daa20>),
  datetime.timedelta(microseconds=198446)),
 (('#job_dept', <data_structs.Expression at 0x1c0bd271860>),
  datetime.timedelta(microseconds=164560)),
 (('#weight', <data_structs.Expression at 0x1c0a5c472e8>),
  datetime.timedelta(microseconds=146608)),
 (('#grad_time', <data_structs.Expression at 0x1c0abcc1438>),
  datetime.timedelta(microseconds=141620)),
 (('#race', <data_structs.Expression at 0x1c0a5d48588>),
  datetime.timedelta(microseconds=140624)),
 (('#edu_time', <data_structs.Expression at 0x1

In [77]:
%prun sr.compile_tokens_to_expression('欧 *阳|独 *孤|令 *狐|皇 *甫|夏 *侯|呼 *延|诸 *葛|完 *颜|拓 *跋|公 *孙|宇 *文|北 *野|欧 *文|上 *官|端 *木')

         10365420 function calls (10263522 primitive calls) in 3.848 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     2379    0.578    0.000    2.678    0.001 boolean.py:1287(absorb)
  1489490    0.464    0.000    0.600    0.000 boolean.py:892(__eq__)
1214042/1213655    0.412    0.000    0.596    0.000 boolean.py:728(__eq__)
  4291588    0.403    0.000    0.404    0.000 {built-in method builtins.isinstance}
   269622    0.390    0.000    1.026    0.000 boolean.py:1132(__contains__)
   276616    0.304    0.000    0.591    0.000 boolean.py:935(__init__)
 4059/503    0.279    0.000    3.811    0.008 boolean.py:1142(simplify)
   262640    0.257    0.000    0.822    0.000 boolean.py:1030(__init__)
   370126    0.170    0.000    0.472    0.000 {built-in method builtins.all}
   591619    0.120    0.000    0.153    0.000 boolean.py:941(<genexpr>)
   245480    0.095    0.000    0.120    0.000 boolean.py:1065(cancel)
   164679   

In [ ]:
处理 Match_query is True 的情况